In [1]:
import numpy as np
import torch
from sklearn import ensemble, metrics
from lib import data_loader as dl

In [2]:
forest = ensemble.RandomForestClassifier(n_estimators = 100)
train_x, train_y, test_x, test_y = dl.LoadDataNumpy('data')
forest.fit(train_x, train_y)
test_y_predicted = forest.predict(test_x)
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
print('Accuracy =', accuracy)

Accuracy = 0.9053


In [4]:
val_x = dl.GetValidationVectors('data')
val_y_predicted = forest.predict(val_x)
fn = 'data/prediction/val_y_predicted.npy'
np.save(fn, val_y_predicted)
print('data successfully saved at',fn)

data successfully saved at data/prediction/val_y_predicted.npy
